In [1]:
import os, pathlib, shutil
from collections import Counter
from matplotlib import pyplot as plt
from skimage import io
from torch.utils.data import Dataset, DataLoader, Subset, random_split
from torchvision.datasets import ImageFolder
from torchvision.transforms import v2
import numpy as np
import torch
import train

# This is example, modify the directory based on the set up.
src = "D:/599DL4VProject/the_wildfire_dataset"

wf1TrainPath = pathlib.Path(src + '/train').resolve()
wf1ValidPath = pathlib.Path(src + '/valid').resolve()
wf1TestPath = pathlib.Path(src + '/test').resolve()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/nix/store/k7xkpa8lwkjanf79r0fj3bhviv4vqm8q-python3-3.10.13-env/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/nix/store/k7xkpa8lwkjanf79r0fj3bhviv4vqm8q-python3-3.10.13-env/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs m

Random Idea: What if we train and evaluate using batch of very small random crops (64x64) for each image then adjust loss via bayesian or expected value? Actual set of feature needed to classify a sample can be surprisingly small.

In [2]:
p = 0.3
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

tfset = v2.Compose([
        v2.ToImageTensor(),
        v2.ConvertImageDtype(torch.uint8),
        #v2.RandomRotation(degrees=(0,90)),
        v2.RandomCrop(size=512, pad_if_needed=True),
        v2.RandomApply(transforms=[
            v2.RandomInvert(p),
            v2.RandomGrayscale(p),
            v2.ColorJitter(p,p,p,p),
            v2.RandomEqualize(p),
            v2.RandomHorizontalFlip(p),
        ], p=0.6),
        v2.Resize(size=128, antialias=True),
        v2.ConvertImageDtype(dtype=torch.float),
        v2.Normalize(mean, std)
])

wf1Train = train.ForestFireDataset(root=str(wf1TrainPath), transform=tfset)
wf1Valid = train.ForestFireDataset(root=str(wf1ValidPath), transform=tfset)
wf1Test = train.ForestFireDataset(root=str(wf1TestPath), transform=tfset)

#wf1fire = Subset(wf1Train, [i for i, label in enumerate(wf1Train.targets) if label == 1])
wf1nofire = Subset(wf1Train, [i for i, label in enumerate(wf1Train.targets) if label == 0])

batch_size = 16
wf1TrLoader = DataLoader(dataset=wf1nofire, batch_size=batch_size, shuffle=True)
wf1VaLoader = DataLoader(dataset=wf1Valid, batch_size=batch_size)
wf1TsLoader = DataLoader(dataset=wf1Test, batch_size=batch_size)

In [ ]:
# Analysis
print("Training Set: ", Counter(wf1Train.targets))
print("Valid Set: ", Counter(wf1Valid.targets))
print("Test Set: ", Counter(wf1Test.targets))

In [3]:
epochs = 5
iter = 1
trainer = train.GANProject(img_size=128, debug=False, small=True)
trainer.train_model(trainLoader=wf1TrLoader, validLoader=wf1VaLoader, metric='loss', epochs=epochs)

Epoch 0 Time: 847.25s
Train:  {'rec_loss': 0.6942729375133776, 'gen_loss': 0.4967227047436858, 'd_loss': 1.523321616812928, 'feature_loss': 4.121257290774829}
Validation:  {'rec_loss': 3.032583056397699, 'gen_loss': 0.45488234183088466, 'd_loss': 1.6158791916880442}


: 

: 

In [ ]:
t_epoch = 9
gen_path = pathlib.Path(f'genproject_e{t_epoch}.pt').resolve()
dsc_path = pathlib.Path(f'dscproject_e{t_epoch}.pt').resolve()
gen = torch.load(gen_path, map_location=device)
dsc = torch.load(dsc_path, map_location=device)
predictor = train.GANProject(load_unet=gen, load_cnn=dsc, img_size=128, debug=False)

In [ ]:
test_result = predictor.ganEval_epoch(wf1TsLoader)
print(test_result)